In [3]:
import numpy as np
import torch
torch.set_printoptions(sci_mode=False)
import networkx as nx
from netAPI import adjacency_matrix

from GNNAPI import buildVNNConfig
# from GNNAPI import EarlyStopper
from torch_geometric.nn import GATConv, GCNConv
#from NetRepLearnerV2 import NetGNNAttRepr
from NetRepLearner_mh import NetReprLearning
# from NetRepLearner_mh import GMLearning
import pandas as pd
from netAPI import pickleLoad, pickleDump
# from GNNAPI import nnSquare, nnExp
from GNNAPI import VNN_MLP
from GNNAPI import GNN_VNN_Layer
from GNNAPI import buildLaplacian
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist,pdist
import scipy.stats
# from GNNAPI import DirectTensor
import scipy.optimize
import scipy.stats as stats

from netAPI import loadNetworkMat
from netAPI import adjacency_matrix
from GNNAPI import GNN_VNN_Layer
from GNNAPI import VNNDefaultConfig
from GNNAPI import buildLaplacian
from GNNAPI import buildVNNConfig
# from GNNAPI import matrixNormalize
# from GNNAPI import GNN_VNN_Multiclass_Layer
from GNNAPI import NNmodel
from GNNAPI import dmerge
import time

import geopandas as gpd
from shapely.geometry import Point
from esda.moran import Moran
from esda.geary import Geary
import libpysal

from sklearn.preprocessing import StandardScaler

import torch.nn as nn
import matplotlib.pyplot as plt
import scipy

import torch.nn.functional as F
import torch.optim as optim
from visdom import Visdom
import warnings
from csv import writer
warnings.filterwarnings('ignore')
from sklearn.mixture import GaussianMixture


In [4]:
def inbetween(o,d,A,win,wout):
    distance = A[o,d]
    
    if win[A[o,:]<distance].shape[0]>0: 
        # number of available jobs in a shorter distance
        ofts = win[A[o,:]<distance].sum(axis=0,keepdims=True)[0]
    else:
        ofts = 0
    if wout[A[d,:]<distance].shape[0]>0: 
        # number of available residences in a shorter distance
        dfts = wout[A[d,:]<distance].sum(axis=0,keepdims=True)[0]
    else:
        dfts = 0
#     print(ofts,dfts,win[d],wout[o])
    ofts = ofts/(win[d]+1)
    dfts = dfts/(wout[o]+1)
    between_fts = torch.FloatTensor([ofts,dfts,distance]).view(1,3)
#     print(between_fts)
    return between_fts


In [ ]:

splitByNodes = True
externalities = True
# if modelmode='g', conventional singly constrained gravity model, other parameters will be ignored
modelmode = 'p'
# gravi con, modelmode = 'p', attention = True, directEmebdding = False
# GCN + MLP, modelmode = 'p', attention = False, VNNattraction = Truepath = 'LEHD/'

cities = [
    ('New York City', 'ny', ['New York County, NY', 'Queens County, NY','Kings County, NY','Bronx County, NY','Richmond County, NY']),
    ('Los Angeles', 'ca', ['Los Angeles County, CA']),
    ('Chicago', 'il', ['Cook County, IL']),
    ('Houston', 'tx', ['Harris County, TX']),
    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
]

splitByNodes = True
externalities = True
# if modelmode='g', conventional singly constrained gravity model, other parameters will be ignored
modelmode = 'p'
# gravi con, modelmode = 'p', attention = True, directEmebdding = False
# GCN + MLP, modelmode = 'p', attention = False, VNNattraction = True
attention = True
VNNattraction = False
directEmebdding = False #learn node embedding directly (MLP) or through GNNs

for city,state, counties in cities:

   
    print('processing LEHD')
    LEHD = pd.read_csv(f'LEHD/{city}_flow.csv')
    nodes = list(set(LEHD.O.unique().tolist()+LEHD.D.unique().tolist()))
    G = nx.DiGraph()    ('Boston', 'ma', ['Suffolk County, MA', 'Middlesex County, MA']),
    ('Phoenix', 'az', ['Maricopa County, AZ']),
    ('Philadelphia', 'pa', ['Philadelphia County, PA']),
    ('San Antonio', 'tx', ['Bexar County, TX']),
    ('San Diego', 'ca', ['San Diego County, CA']),
    ('Dallas', 'tx', ['Dallas County, TX']),
    ('San Jose', 'ca', ['Santa Clara County, CA']),
    ('Austin', 'tx', ['Travis County, TX']),
    G.add_weighted_edges_from([(LEHD.iloc[e]['O'], LEHD.iloc[e]['D'],
                            LEHD.iloc[e]['Volume']) for e in range(len(LEHD))])
    #main parameters
    mainepochs = 100000
    ed = 16 #node embedding dimensionality
    splitSeed = 0
    seed = 0


    #define the adjacency matrix
    OD = 1.0 * np.array(nx.adjacency_matrix(G,sorted(list(G.nodes()))).todense())
    wind = (OD.sum(axis = 0) > 0) & (OD.sum(axis = 1) > 0)
    location = read_location_from_ct(state,nodes)
    A = scipy.spatial.distance_matrix(location,location)
    n = A.shape[0]

    train,valid, test = getTrainTestbyNodes(A,train_p= 0.7,  seed = splitSeed)
    node_train = train
    node_valid = valid
    node_test = test
    train = np.full(A.shape,True)*train*(train.reshape(-1,1))
    valid = np.full(A.shape,True)*valid*(valid.reshape(-1,1))
    test = np.invert(train+valid)

    if externalities:
        print('processing POI')
        nodefts = pd.read_csv('LEHD/'+city+'fts.csv')
        nodefts = nodefts.set_index('GEOID').loc[sorted(list(G.nodes()))].values
    else:
        nodefts = torch.FloatTensor(np.eye(n))

    nodefts = torch.FloatTensor(nodefts)
    nodefts_train = nodefts[node_train]
    #some basic stats
    print('City {}, Network of size {}, non-zero edges = {}, unique edge values = {}, avg. edge = {},total network weight = {}'.format(city, len(G), len(G.edges()), len(np.unique(A)), A.mean(), A.sum()))

    A_train = A[:,node_train][node_train]
    nodefts_train = nodefts[node_train]
    OD_train = OD[:,node_train][node_train]
#     location_train = location[node_train]
    edges_train = A_train.nonzero()
    wout_train = OD_train.sum(axis = 1)
    win_train = OD_train.sum(axis = 0)
    for i,o in enumerate(edges_train[0]):
        d = edges_train[1][i]
        if o!=d:
            if i == 0:
                between_fts_train = inbetween(o,d,A_train,win_train,wout_train)
            else:
                fts_temp = inbetween(o,d,A_train,win_train,wout_train)
                between_fts_train = torch.concat([between_fts_train,fts_temp],axis=0)
            if i%100 == 0 and i >100:
                progress_bar(i,len(edges_train[0]))

    A_valid = A[:,node_valid][node_valid]
    nodefts_valid = nodefts[node_valid]
    OD_valid = OD[:,node_valid][node_valid]
    wout_valid = OD_valid.sum(axis = 1)
    win_valid = OD_valid.sum(axis = 0)
    edges_valid = A_valid.nonzero()
    for i,o in enumerate(edges_valid[0]):
        d = edges_valid[1][i]
        if o!=d:
            if i == 0:
                between_fts_valid = inbetween(o,d,A_valid,win_valid,wout_valid)
            else:
                fts_temp = inbetween(o,d,A_valid,win_valid,wout_valid)
                between_fts_valid = torch.concat([between_fts_valid,fts_temp],axis=0)
            if i%100 == 0 and i >100:
                progress_bar(i,len(edges_valid[0]))
    torch.save(nodefts_train, 'training/'+city+'_nodefts_train.pt')
    torch.save(nodefts_valid, 'training/'+city+'_nodefts_valid.pt')

    torch.save(between_fts_train, 'training/'+city+'_between_fts_train.pt')
    torch.save(between_fts_valid, 'training/'+city+'_between_fts_valid.pt')
    
    A_train = torch.FloatTensor(A_train)
    A_valid = torch.FloatTensor(A_valid)
    
    torch.save(A_train, 'training/'+city+'_A_train.pt')
    torch.save(A_valid, 'training/'+city+'_A_valid.pt')
    
    torch.save(OD_train, 'training/'+city+'_OD_train.pt')
    torch.save(OD_valid, 'training/'+city+'_OD_valid.pt')
    

    A_test = A[:,node_test][node_test]

    nodefts_test = nodefts[node_test]

    OD_test = OD[:,node_test][node_test]
    OD_test = torch.FloatTensor(OD_test)
    win_test = OD_test.sum(axis = 0)
    wout_test = OD_test.sum(axis = 1)

    edges_test = A_test.nonzero()
    for i,o in enumerate(edges_test[0]):
        d = edges_test[1][i]
        if o!=d:
            if i == 0:
                between_fts_test = inbetween(o,d,A_test,win_test,wout_test)
            else:
                fts_temp = inbetween(o,d,A_test,win_test,wout_test)
                between_fts_test = torch.concat([between_fts_test,fts_temp],axis=0)
            if i%100 == 0 and i >100:
                progress_bar(i,len(edges_test[0]))

    A_test = torch.FloatTensor(A_test).to(torch.device("cuda"))

    torch.save(A_test, 'training/'+city+'_A_test.pt')
    torch.save(OD_test, 'training/'+city+'_OD_test.pt')
    torch.save(nodefts_test, 'traini8ng/'+city+'_nodefts_test.pt')
    torch.save(between_fts_test, 'training/'+city+'_between_fts_test.pt')
